### Biblioteca Numpy
Numpy adiciona a linguagem Python facilidades para a manipulação e cálculos em arrays de grande dimensão. Ela adiciona o tipo de dado "ndarray", tipo que a epmwebapi utiliza para o retorno de séries temporais do EPM Server.

**Conexão ao EPM Server e busca de dados**

In [1]:
import epmwebapi as epm
import numpy as np
import datetime

# Método para não expor usuário e senha do EPM no código fonte
import getpass
user = input('EPM user:')
password = getpass.getpass("EPM password:")

# Crie o objeto de conexão informando os endereços do EPM Webserver(Authentication Port e WEB API Port), usuário e senha.
connection = epm.EpmConnection('https://localhost:44340', 'https://localhost:44339', user, password)

bvname = 'MemoriaEpmServer'

bv = connection.getDataObjects([bvname])

end_date = datetime.datetime.now()
ini_date= end_date - datetime.timedelta(minutes=10)
queryPeriod = epm.QueryPeriod(ini_date,end_date)

data = bv[bvname].historyReadRaw(queryPeriod)

# Fechando a conexão
connection.close()

EPM user: sa
EPM password: ········


In [2]:
# As funções de leitura da epmwebapi retornam um ndarray (n dimentional array) com três campos: Timestamp, Value e Quality.
print(data)

[(125720., datetime.datetime(2025, 10, 9, 13, 15, 6, 425000, tzinfo=datetime.timezone.utc), 0)
 (122648., datetime.datetime(2025, 10, 9, 13, 15, 16, 451000, tzinfo=datetime.timezone.utc), 0)
 (123524., datetime.datetime(2025, 10, 9, 13, 20, 47, 512000, tzinfo=datetime.timezone.utc), 0)
 (125904., datetime.datetime(2025, 10, 9, 13, 20, 57, 540000, tzinfo=datetime.timezone.utc), 0)
 (126168., datetime.datetime(2025, 10, 9, 13, 22, 37, 845000, tzinfo=datetime.timezone.utc), 0)]


**Índices**

Os índices começam em zero. E podem ser usados indices negativos, começando em -1, para percorrer o ndarray de trás para frente.

In [3]:
# Para acessar os campos use colchetes[]:
data['Value']

array([125720., 122648., 123524., 125904., 126168.], dtype='>f4')

In [4]:
# Acessando o primeiro valor do ndarray 
print(data['Value'][0])

# Acessando o terceiro valor
print(data['Value'][2])

# Acessando o último valor do ndarray 
print(data['Value'][-1])

# Acessando o penúltimo valor do ndarray 
print(data['Value'][-2])

125720.0
123524.0
126168.0
125904.0


 **Fatiamentos**
 
 Fatiar é a forma de escolher valores dentro do ndarray. Usamos a sintaxe **ndarray[start : stop : step]**

In [5]:
# Imprimindo novamente o array para termos a referência de valores
print(data['Value'])

# Acessando os dois primeiros elementos
print(data['Value'][0:2])

# Acessando o terceiro e quarto elemento
print(data['Value'][2:4])

# Acessando os elementos de índice impar
print(data['Value'][1::2])

[125720. 122648. 123524. 125904. 126168.]
[125720. 122648.]
[123524. 125904.]
[122648. 125904.]


**Cálculos**

O Numpy possui diversos métodos que retornam valores cálculados do array.

In [6]:
# Using NumPy's built-in functions to Find Mean, Median, SD and Variance
mean = np.mean(data['Value'])
median = np.median(data['Value'])
sd = np.std(data['Value'])
variance = np.var(data['Value'])

# Printing the values
print("Mean", mean) 
print("Median", median) 
print("Standard Deviation", sd) 
print("Variance", variance) 

Mean 124792.8
Median 125720.0
Standard Deviation 1427.9819
Variance 2039132.2


### Exemplos práticos

**Remoção de outliers**

Utilizamos a função **percentile** que retorna os quartis, para usar o método descrito aqui: https://www.purplemath.com/modules/boxwhisk3.htm

In [7]:
temperaturas = [ 25, 26, 225, 24, 23, 24, 25, 325, 28, 27]

def removeoutlier(values):
    fator = 1.5
    q3, q1 = np.percentile(values, [75, 25])

    iqr = q3 - q1

    lowpass = q1 - (iqr * fator)

    highpass = q3 + (iqr * fator)
    
    return [v for v in values if v > lowpass and v < highpass]

data = removeoutlier(temperaturas)
print(data)

[25, 26, 24, 23, 24, 25, 28, 27]


**Para mais informações acesse a documentação da biblioteca**

https://docs.scipy.org/doc/numpy/reference/routines.html
